In [7]:
import sys
sys.path.append('../')
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import lightning.pytorch as pl
import lightning.pytorch.callbacks as plc
import torch.nn.functional as F
import torch.nn as nn
import os
import sys
import pytorch_lightning.loggers as pl_loggers
import pickle
import math
import json
from test_case import SeqFormer
from test_case import get_plan_encoding, configs, load_json, get_op_name_to_one_hot, plan_parameters, add_numerical_scalers
from leon_experience import Experience
import numpy as np
import ray
import time
from argparse import ArgumentParser
import copy
import wandb
import time
import random


import pickle
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'

In [2]:
with open('../log/exp_v2.pkl', 'rb') as f:
    exp = pickle.load(f)

In [6]:
def Getpair(exp):
    pairs = []
    for eq in exp.keys():
        for j in exp[eq]:
            for k in exp[eq]:
                if (j[0].info['sql_str'] == k[0].info['sql_str']) and (j[0].hint_str() == k[0].hint_str()): # sql 和 hint 都相同
                    continue
                if j[0].info['sql_str'] != k[0].info['sql_str'] and (j[0].info['latency'] == 90000 or k[0].info['latency'] == 90000):
                    continue
                # if (j[0].info['latency'] == k[0].info['latency']): # latency 相同 1s之内不把他train_pair
                if max(j[0].info['latency'],k[0].info['latency']) / min(j[0].info['latency'],k[0].info['latency']) < 1.2:
                    continue
                # if j[0].info['latency'] == 90000 or k[0].info['latency'] == 90000:
                #     continue
                tem = [j, k]
                pairs.append(tem)
    return pairs

In [8]:
from util.dataset import *
from util.model import *
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'
Transformer_model = SeqFormer(
                        input_dim=configs['node_length'],
                        hidden_dim=256,
                        output_dim=1,
                        mlp_activation="ReLU",
                        transformer_activation="gelu",
                        mlp_dropout=0.1,
                        transformer_dropout=0.1,
                    )
prev_optimizer_state_dict = None
model = Transformer_model.to(DEVICE)
model = PL_Leon(model, prev_optimizer_state_dict)
train_pairs = Getpair(exp)
leon_dataset = prepare_dataset(train_pairs)
dataloader_train = DataLoader(leon_dataset, batch_size=256, shuffle=True, num_workers=0)
# dataloader_val = DataLoader(leon_dataset, batch_size=256, shuffle=False, num_workers=0)
dataset_val = BucketDataset(exp, keys=exp.keys())
batch_sampler = BucketBatchSampler(dataset_val.buckets, batch_size=1)
dataloader_val = DataLoader(dataset_val, batch_sampler=batch_sampler)
# model = load_model(model_path, prev_optimizer_state_dict).to(DEVICE)
model.optimizer_state_dict = prev_optimizer_state_dict
current_directory = os.getcwd()
# 获取上一级目录
parent_directory = os.path.dirname(current_directory)
logger =  pl_loggers.WandbLogger(save_dir=parent_directory + '/logs', name="exp_pair", project='leon3')
trainer = pl.Trainer(accelerator="gpu",
                    devices=[2],
                    max_epochs=10,
                    logger=logger)
trainer.fit(model, dataloader_train, dataloader_val)
prev_optimizer_state_dict = trainer.optimizers[0].state_dict()
print("*"*20)
print("Current Accuracy For Each EqSet: ", model.eq_summary)
print("*"*20)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/data1/wyz/miniconda3/envs/leon/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/data1/wyz/miniconda3/envs/leon/lib/python3.8/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory /data1/wyz/online/LEONForPostgres/logs/leon3/8xbhajas/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name  | Type      | Params
------------------------------------
0 | model | SeqFormer | 48.8 K
------------------------------------
48.8 K    Trainable params
0         Non-trainable params
48.8 K    Total params
0.195     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 1472/1472 [00:52<00:00, 27.78it/s, v_num=ajas]   

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 1472/1472 [00:53<00:00, 27.77it/s, v_num=ajas]
********************
Current Accuracy For Each EqSet:  {'ci,cn,k,mc,mk,t': (0.9175704989154013, 461), 'ci,cn,k,mc,mk,n,t': (0.9232693911592994, 1199), 'cc,cct1,cct2,ci,k,kt,mk,t': (0.8498402555910544, 313), 'ci,it1,it2,k,mi,mi_idx,mk,n,t': (0.9515545914678236, 1383), 'an1,ci,cn,mc,n1,rt,t': (0.8, 120), 'a1,ci,cn,mc,n1,rt,t': (0.7796610169491526, 59), 'cn,ct,it1,it2,mc,mi,mi_idx,t': (0.6201834862385321, 545), 'cc,cct1,cct2,ci,it1,it2,k,mi,mi_idx,mk,n,t': (0.9167008617152236, 2437), 'an,ci,k,mc,mk,n': (0.5454545454545454, 11), 'an,cc,cct1,cct2,chn,ci,cn,it,it3,k,mc,mi,mk,n,pi,rt,t': (0.8097269624573379, 1172), 'ci,k,mc,mk,n,t': (0.9015151515151515, 660), 'ci,it1,it2,mi,mi_idx': (0.15625, 32), 'cc,cct1,cct2,chn,ci,k,kt,mk,t': (0.8461538461538461, 533), 'ct,it,kt,mc,miidx,t': (1.0, 64), 'cn,it1,it2,mc,mi,mi_idx,t': (0.838150289017341, 346), 'cn,ct,it1,k,kt,mc,mi,mi_idx,mk,t': (0.9018733273862622, 1121), 'cn,ct,k,lt,mc